In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline

import pickle 

In [22]:
# load data
data = pd.read_csv('Landsat_training_farms_Tuned.csv')

data.drop(['system:index','system:time_start','system:time_start_1','.geo'], axis=1, inplace=True)

y = data['CID']
X = data.drop(['CID'], axis=1)

In [23]:
def trainModel(X,y):  
       
    #Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=y, random_state=123)
        
    # instantiate model
    model = RandomForestClassifier(n_estimators=250)
    
    # Modeling code in a pipeline
    my_pipeline = Pipeline(steps=[('model', model)])

    # fit model on data:
    model = my_pipeline.fit(X_train, y_train)
    
    # save the model to disk
    filename = 'trained_model_landsatTuned.sav'
    #pickle.dump(my_pipeline, open(filename, 'wb'))
    
    return pickle.dump(model, open(filename, 'wb'))

In [24]:
trained_rf_model = trainModel(X,y)

In [18]:
def predictCrop(array):

    """ This function loads model from disk and predicts
    the probability of copy type.
    input: array of the 13 Sentinel bands, QA10,20 and 60,
    ndvi, and season."""

    # load the model from disk
    filename = 'trained_model_landsatTuned.sav'
    trained_model = pickle.load(open(filename, 'rb'))

    # Predict crop type
    predicted_crop_type = trained_model.predict_proba([array])

    return predicted_crop_type